
# 📘 Lecture 2

## Direct Elimination — Cost of LU vs Gauss–Jordan

This in-class hands-on exercise compares the computational work required for **LU factorization + triangular solves** versus **Gauss–Jordan elimination** when solving the same matrix system with many different right-hand sides.


## --- Google Colab environment setup ---

The cell below only needs to run when the notebook is opened in Google Colab.

This code will not affect code execution locally in VS-code + conda environment.

Google Colab starts each session with its own **preloaded versions** of common Python (currently 3.12.12) and Python packages (NumPy, SciPy, etc.).  
If we install different package versions once loaded, Python cannot switch to them while it is already running.

### What will happen
When you run the setup cell below in Google Colab:

1. The required package versions are installed
2. The runtime is **automatically restarted** so the new versions can be loaded  
3. You may see the message **“Your session crashed for an unknown reason.”**  
   → This is expected and normal

After the restart, rerun the notebook and check the **version check cell** to confirm package versions are correct.

### Runtime menu notes
- **Runtime → Restart session**  
  Restarts Python but keeps installed packages and saved files

- **Runtime → Disconnect and delete runtime**  
  Resets Colab completely to its default environment (packages will need to be reinstalled)


In [2]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])


Not running in Google Colab — setup skipped.
Python version: 3.12.12


In [1]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)


numpy: 2.4.0
scipy: 1.16.3
matplotlib: 3.10.8
pandas: 2.3.3


---


## Counting computational work (not timing)

We will compare two strategies for solving $\mathbf{A}\mathbf{x}=\mathbf{b}$ for **many** different right-hand sides $\mathbf{b}$:

- **LU approach**: factorize once, then solve each new $\mathbf{b}$ with forward + back substitution.
- **Gauss–Jordan approach**: redo elimination on $[\mathbf{A}\,|\,\mathbf{b}]$ for every new $\mathbf{b}$.

Instead of measuring wall-clock time (which depends on hardware and libraries), we will count basic floating-point operations:

- additions/subtractions
- multiplications
- divisions


In [1]:
import numpy as np
from dataclasses import dataclass


@dataclass
class OpCount:
    add: int = 0
    mul: int = 0
    div: int = 0

    def total(self) -> int:
        return self.add + self.mul + self.div

    def __iadd__(self, other):
        self.add += other.add
        self.mul += other.mul
        self.div += other.div
        return self


def lu_factor_count(A: np.ndarray, pivot: bool = True):
    """Educational LU (Doolittle) with partial pivoting. Counts floating ops."""
    A = A.astype(float).copy()
    n = A.shape[0]
    P = np.arange(n)

    L = np.eye(n)
    U = A.copy()

    ops = OpCount()

    for k in range(n - 1):
        if pivot:
            pivot_row = k + np.argmax(np.abs(U[k:, k]))
            if abs(U[pivot_row, k]) < 1e-14:
                raise ValueError("Singular or near-singular matrix.")
            if pivot_row != k:
                U[[k, pivot_row], :] = U[[pivot_row, k], :]
                L[[k, pivot_row], :k] = L[[pivot_row, k], :k]
                P[[k, pivot_row]] = P[[pivot_row, k]]

        pivot_val = U[k, k]
        for i in range(k + 1, n):
            ops.div += 1
            m = U[i, k] / pivot_val
            L[i, k] = m

            for j in range(k, n):
                ops.mul += 1
                ops.add += 1
                U[i, j] -= m * U[k, j]

            U[i, k] = 0.0

    return P, L, U, ops


def forward_sub_count(L: np.ndarray, b: np.ndarray):
    """Solve L y = b (lower triangular). Counts floating ops."""
    n = L.shape[0]
    y = np.zeros(n)
    ops = OpCount()

    for i in range(n):
        s = 0.0
        for j in range(i):
            ops.mul += 1
            ops.add += 1
            s += L[i, j] * y[j]
        ops.add += 1
        rhs = b[i] - s
        ops.div += 1
        y[i] = rhs / L[i, i]
    return y, ops


def back_sub_count(U: np.ndarray, y: np.ndarray):
    """Solve U x = y (upper triangular). Counts floating ops."""
    n = U.shape[0]
    x = np.zeros(n)
    ops = OpCount()

    for i in range(n - 1, -1, -1):
        s = 0.0
        for j in range(i + 1, n):
            ops.mul += 1
            ops.add += 1
            s += U[i, j] * x[j]
        ops.add += 1
        rhs = y[i] - s
        ops.div += 1
        x[i] = rhs / U[i, i]
    return x, ops


def gauss_jordan_solve_count(A: np.ndarray, b: np.ndarray, pivot: bool = True):
    """Solve Ax=b via Gauss–Jordan on [A|b]. Counts floating ops."""
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = A.shape[0]

    Ab = np.hstack([A, b.reshape(-1, 1)])
    ops = OpCount()

    for k in range(n):
        if pivot:
            pivot_row = k + np.argmax(np.abs(Ab[k:, k]))
            if abs(Ab[pivot_row, k]) < 1e-14:
                raise ValueError("Singular or near-singular matrix.")
            if pivot_row != k:
                Ab[[k, pivot_row], :] = Ab[[pivot_row, k], :]

        pivot_val = Ab[k, k]
        for j in range(k, n + 1):
            ops.div += 1
            Ab[k, j] /= pivot_val

        for i in range(n):
            if i == k:
                continue
            factor = Ab[i, k]
            for j in range(k, n + 1):
                ops.mul += 1
                ops.add += 1
                Ab[i, j] -= factor * Ab[k, j]
            Ab[i, k] = 0.0

    x = Ab[:, -1]
    return x, ops


## Compare work for many right-hand sides

We generate one matrix $\mathbf{A}$, then solve for multiple different vectors $\mathbf{b}$.

Try changing:
- `n` (matrix size)
- `rhs_count` (number of right-hand sides)


In [2]:
def compare_costs(n=30, rhs_count=20, seed=1):
    rng = np.random.default_rng(seed)

    # Build a well-conditioned-ish matrix, then perturb to make it general (slightly non-symmetric)
    M = rng.standard_normal((n, n))
    A = M.T @ M + np.eye(n) * n
    A = A + 0.05 * rng.standard_normal((n, n))

    Bs = rng.standard_normal((n, rhs_count))

    # LU: factor once
    P, L, U, ops_lu_fact = lu_factor_count(A, pivot=True)

    ops_lu_total = OpCount()
    ops_lu_total += ops_lu_fact

    # Solve each RHS with forward + back substitution
    for r in range(rhs_count):
        b = Bs[:, r]
        b_perm = b[P]
        y, ops_f = forward_sub_count(L, b_perm)
        _, ops_b = back_sub_count(U, y)
        ops_lu_total += ops_f
        ops_lu_total += ops_b

    # Gauss–Jordan: redo full elimination for each RHS
    ops_gj_total = OpCount()
    for r in range(rhs_count):
        b = Bs[:, r]
        _, ops_gj = gauss_jordan_solve_count(A, b, pivot=True)
        ops_gj_total += ops_gj

    return ops_lu_fact, ops_lu_total, ops_gj_total

In [10]:
# --- Try changing these ---
n = 30
rhs_count = 30

ops_lu_fact, ops_lu_total, ops_gj_total = compare_costs(n=n, rhs_count=rhs_count, seed=1)

print(f"n={n}, number of RHS={rhs_count}\n")

print("LU approach:")
print(f"  factorization ops: add={ops_lu_fact.add}, mul={ops_lu_fact.mul}, div={ops_lu_fact.div}, total={ops_lu_fact.total()}")
print(f"  total (factor + all solves): add={ops_lu_total.add}, mul={ops_lu_total.mul}, div={ops_lu_total.div}, total={ops_lu_total.total()}\n")

print("Gauss–Jordan approach (redo each RHS):")
print(f"  total ops: add={ops_gj_total.add}, mul={ops_gj_total.mul}, div={ops_gj_total.div}, total={ops_gj_total.total()}\n")

ratio = ops_gj_total.total() / ops_lu_total.total()
print(f"Operation-count ratio (GJ / LU_total) ≈ {ratio:.2f}×")


n=30, number of RHS=30

LU approach:
  factorization ops: add=8990, mul=8990, div=435, total=18415
  total (factor + all solves): add=36890, mul=35090, div=2235, total=74215

Gauss–Jordan approach (redo each RHS):
  total ops: add=430650, mul=430650, div=14850, total=876150

Operation-count ratio (GJ / LU_total) ≈ 11.81×


## Small matrix demo (lecture example)

This is a tiny 3×3 example. The important takeaway is not the exact number, but the pattern:

- LU: factor once, then reuse for many $\mathbf{b}$
- Gauss–Jordan: redo elimination for each $\mathbf{b}$


In [12]:
A = np.array([[4.0, 1.0, 2.0],
              [1.0, 3.0, 0.0],
              [2.0, 0.0, 5.0]])

rhs_count = 10
rng = np.random.default_rng(0)
Bs = rng.standard_normal((3, rhs_count))

P, L, U, ops_lu_fact = lu_factor_count(A, pivot=True)

ops_lu_total = OpCount(); ops_lu_total += ops_lu_fact
for r in range(rhs_count):
    b = Bs[:, r]
    y, ops_f = forward_sub_count(L, b[P])
    _, ops_b = back_sub_count(U, y)
    ops_lu_total += ops_f; ops_lu_total += ops_b

ops_gj_total = OpCount()
for r in range(rhs_count):
    _, ops_gj = gauss_jordan_solve_count(A, Bs[:, r], pivot=True)
    ops_gj_total += ops_gj

print(f"A is 3x3, RHS count={rhs_count}\n")
print("LU total ops (factor + solves):", ops_lu_total.total())
print("Gauss–Jordan total ops:", ops_gj_total.total())
print("ratio (GJ / LU_total):", ops_gj_total.total() / ops_lu_total.total())


A is 3x3, RHS count=10

LU total ops (factor + solves): 259
Gauss–Jordan total ops: 450
ratio (GJ / LU_total): 1.7374517374517375


## 📝 In-Class Practice

1. Increase `n` from 30 → 60 → 120 and keep `rhs_count` fixed. What happens to the ratio?
2. Fix `n`, then increase `rhs_count` from 1 → 5 → 20 → 100. What happens to the ratio?
3. Explain (in words) why LU benefits from repeated solves but Gauss–Jordan does not.
